## 도구
에이전트, 체인 또는 LLM이 외부 세계와 상호작용하기 위한 인터페이스.
LangChain에서 기본 제공하는 도구를 사용하여 쉽게 활용할 수 있고, 사용자 정의 도구를 만들어 활용할 수 있다.


In [1]:
from dotenv import load_dotenv
import warnings

load_dotenv()
warnings.filterwarnings("ignore")


## 빌트인 도구(Built-in Tools)
랭체인에서 제공하는 사전에 정의된 도구(tool)와 툴킷(toolkit) 사용 가능.
tool은 단일 도구를 의미, toolkit은 여러 도구를 모아놓은 집합체를 의미.

## Python REPL 도구
Python 코드를 REPL(Read-Eval-Print Loop) 환경에서 실행하기 위한 클래스 제공.

- 설명
    - Python 셀 환경 제공
    - 유효한 Python 명령어 입력 받아 실행
    - print(...) 함수 사용
- 주요 특징
    - senitize_input: 입력을 정제하는 옵션(기본값: True)
    - python_repl: pythonREPL 인스턴스(기본값: 전역범위에서 실행)

- 사용 방법
    - Python REPLTool 인스턴스 생성
    - run 또는 arun, invoke 메서드 사용하여 Python 코드 실행

- 입력 정제
    - 입력 문자열에서 불필요한 공백, 백틱, 'python' 키워드 등을 제거.

In [2]:
from langchain_experimental.tools import PythonREPLTool

# 파이썬 코드를 실행하는 도구 생성
python_repl_tool = PythonREPLTool()

In [3]:
# 파이썬 코드 실행하고 결과 반환
result = python_repl_tool.run("print(100 + 200)")
print(result)

Python REPL can execute arbitrary code. Use with caution.


300



In [4]:
# LLM에게 파이썬 코드 작성하도록 요청하고 결과 반환하는 예제
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
 
def print_and_excute(code: str, debug: bool = True) -> str:
    if debug:
        print(f"Code:\n{code}")
    return python_repl_tool.invoke(code)


prompt = ChatPromptTemplate.from_messages([
    (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ('human', "{input}"),
])
llm = ChatOpenAI(model="gpt-4o", temperature=0)
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_excute)

In [5]:
chain.invoke('로또 번호 생성기를 출력하는 코드를 작성하세요.')

Code:
import random

def generate_lotto_numbers():
    return sorted(random.sample(range(1, 46), 6))

print(generate_lotto_numbers())


'[13, 20, 24, 27, 31, 33]\n'

## 검색 API 도구
Tavily 검색 API를 활용하여 검색 기능 구현한 도구. TavilySearchResults, TavilyAnswer 클래스 제공.

### TavilySearchResults

설명
- Tavily 검색 API를 쿼리하고, JSON 형식의 결과를 반환.
- 포괄적이고 정확하며 신뢰할 수 있는 결과에 최적화된 검색 엔진.
- 현재 이벤트에 대한 질문에 답변할 때 유용.

주요 매개변수
- max_results: int: 최대 검색 결과 수(기본값: 10)
- search_depth: str: 검색 깊이('basic' 또는 'advanced')
- include_domains (List[str]): 포함할 도메인 목록
- exclude_domains (List[str]): 제외할 도메인 목록
- include_answer (bool): 원본 쿼리에 대한 짧은 답변 포함 여부
- include_raw_content (bool): 각 사이트의 정제된 HTML 콘텐츠 포함 여부
- include_images (bool): 쿼리 관련 이미지 목록 포함 여부

반환 값
- 검색 결과를 포함하는 JSON 형식의 문자열(url, content)



In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    include_domains = ["github.io", "wikidocs.net"],
)


In [7]:
# 도구 실행
tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요."})

[{'url': 'https://wikidocs.net/262582',
  'content': "LangChain 에서 기본 제공하는 도구를 사용하여 쉽게 도구를 활용할 수 있으며, 사용자 정의 도구(Custom Tool) 를 쉽게 구축하는 것도 가능합니다. LangChain 한국어 튜토리얼\\n바로가기 👀\\n[LangChain] 에이전트(Agent)와 도구(tools)를 활용한 지능형 검색 시스템 구축 가이드\\n2024년 02월 09일\\n41 분 소요\\n이 글에서는 LangChain 의 Agent 프레임워크를 활용하여 복잡한 검색과 📍 전체 템플릿 코드\\n다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\\nLangSmith 추적\\n마무리입니다!\\n 문서 기반 QA 시스템 설계 방법 - 심화편\\n2024년 02월 06일\\n22 분 소요\\nLangChain의 RAG 시스템을 통해 문서(PDF, txt, 웹페이지 등)에 대한 질문-답변을 찾는 과정을 정리하였습니다.\\n'},  {'url': 'https://wikidocs.net/234282',  'content': 'Mar 19, 2024 · langchain 은 언어 모델과 관련된 다양한 기능을 제공하는 라이브러리로, 이 중 검색 도구 생성 기능은 데이터 검색 및 처리 작업을 용이하게 한다.'}] 이 도구를 사용하면 DALL-E API를 쉽게 통합하여 텍스트 기반 이미지 생성 기능을 구현할 수 있습니다."},
 {'url': 'https://teddylee777.github.io/langchain/langchain-agent/',
  'content': '태그:\nAgent,\nAPI KEY,\nFAISS,\nLangChain,\nLangSmith,\nmemory,\nOpenAI,\nPython,\nRetriever,\nTavily Search,\ntools,\n검색도구,\n랭체인,\n에이전트\n카테고리:\nlangchain\n업데이트: 2024년 02월 09일\n참고\n

## Image 생성 도구(DALL-E)
- DallEAPIWrapper 클래스: OpenAI의 DALL-E 이미지 생성기를 위한 래퍼.
DALL-E API를 쉽게 통합하여 텍스트 기반 이미지 생성 기능 구현 가능. 다양한 설정 옵션을 유연하고 강력한 이미지 생성도구로 활용 가능.

주요 속성
- model: 사용할 DALL-E 모델 이름(기본값: "dall-e-2", "dall-e-3")
- n: 생성할 이미지 수 (기본값: 1)
- size: 이미지 크기
    - "dall-e-2": "1024x1024", "512x512", "256x256"
    - "dall-e-3": "1792x1024", "1024x1024", "1024x1792"
- quality: 이미지 품질(기본값: "standard")
    - "standard": 높은 품질
    - "hd": 높은 품질
- style: 이미지 스타일(기본값: "natural")
    - "natural": 자연스러운 이미지
    - "vivid": 선명한 이미지
- max_retries: 최대 재시도 횟수

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

# 체인 생성
chain = prompt | llm | StrOutputParser()

# 체인 실행
image_prompt = chain.invoke({"image_desc": "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"})

print(image_prompt)

A neo-classical painting depicting a group of people in a grand, classical setting, all intently gazing at their smartphones. The scene is rich in detail, featuring marble columns, ornate sculptures, and lush drapery that reflect the grandeur of classical art. The individuals, dressed in elegant, period-appropriate attire, exhibit a mix of expressions ranging from fascination to distraction, highlighting the contrast between their historical clothing and modern technology. Soft, natural light filters through large windows, casting gentle shadows and enhancing the textures of the fabrics and the smooth surfaces of the smartphones. The overall composition should evoke a sense of irony, blending the timelessness of neo-classical art with the contemporary obsession with digital devices.


In [9]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image
import os

# 도구 생성
dalle = DallEAPIWrapper(
    model="dall-e-3",
    size = "1024x1024",
    quality = "standard",
    n = 1,
    )
# 질문
query = "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"

# 이미지 생성 및 URL 받기
# chain.invoke()를 사용하여 이미지 설명을 DALL-E 프롬프트로 변환
# dalle.run()을 사용하여 실제 이미지 생성
image_url = dalle.run(chain.invoke({'image_desc': query}))

# 이미지 표시
Image(url=image_url, width = 500)

## 사용자 정의 도구(Custom Tools)
LangChain에서 제공하는 빌트인 도구 외에도 사용자가 직접 도구를 정의하여 사용 가능.
langchain.tools 모듈에서 제공하는 tool 데코레이터를 사용하여 함수를 도구로 변환.

### @tool 데코레이터
함수를 도구로 변환하는 기능 제공. 다양한 옵션을 통해 도구의 동작을 커스터마이즈 가능.

사용방법
- 함수 위에 @tool 데코레이터 적용
- 필요에 따라 데코레이터 매개변수 설정

In [12]:
from langchain.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [13]:
# 도구 실행
add_numbers.invoke({"a": 1, "b": 2})

3

In [14]:
# 도구 실행
multiply_numbers.invoke({"a": 1, "b": 2})

2

## 구글 뉴스기사 검색 도구
langchain-teddynote 패키지에서 제공하는 GoogleNews 도구를 사용하여 구글 뉴스기사를 검색하는 도구.

설명
- 구글 뉴스 검색 API를 사용하여 최신 뉴스 검색
- 키워드를 기반으로 뉴스를 검색 가능
- 최신 뉴스를 검색할 수 있음.

매개변수
- k(int): 반환할 최대 검색 결과 수(기본값: 5)

In [16]:
from langchain_teddynote.tools import GoogleNews

# 도구 생성
news_tool = GoogleNews()

In [17]:
news_tool.search_latest(k=5)

[{'url': 'https://news.google.com/rss/articles/CBMidEFVX3lxTE9jbU5YMVBvRFJuMW0xSUphbTY2Rnh1X0pfQXJELW1lTzdVSkdNQkJmR0pBYjhKekNqVTJwMHNuOTJIWVQ3bkk5WXc0Y1VfdG5KX0laYkNheGZGY1FlV1RaSU42MnQtY1owTks5ZHl0UjhsYVJ5?oc=5',
  'content': '‘김건희의 벽’ 절감한 한동훈 진퇴양난 - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMid0FVX3lxTFBKSGVfMkxPYl83TWhSWEI0MG1zMlRveUwtdkc0bTlLMjJLenNUZkFiLTF1RW8yTWhtT0tsVFllZks3b3hoa3Mza1JWVE9lR1paU1FOYzdWdHZYMkhTem5rR3dNam05TFJjUzl5UzdtLVVNV2VEb0ow0gFmQVVfeXFMT1FpbmZZU044eXBDLUxLc2xDM1hyLXFKbmV0YVRVWkR0Z3ZtR3ZBYVNkbUJ5NkNXaDdEcjh3R0ZFb0hrcGNKY3FEZE92VS1zeVh3YUNzcjJVV2hSRUlYZlJjZ3I1WlBn?oc=5',
  'content': '여야 27명 ‘명태균 리스트’ 파문…‘여론작업’ 주장에 당사자들 “허위” - 동아일보'},
 {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE9GSHVoUlVtbHhIY3U2ZVJqaHFOaUhXRjFldkZTaDNidWZ3d21mdHdJQ2lCUmw2LVgzM0FKUlRDamt1ZnVPcXl3WDNqOG9OUnBKWnpHUENvYS1Xc0ZKZHJ6NUkyNDkwNjg5M2c?oc=5',
  'content': '[속보] 대통령실 "북한군 즉각 철수 촉구... 북러 \'군사야합\' 좌시 않겠다" - 한국일보'},
 {'url': 'https://news.google.com/rss/artic

In [18]:
news_tool.search_by_keyword(keyword="AI", k=5)

[{'url': 'https://news.google.com/rss/articles/CBMib0FVX3lxTE4yYkp3OHJwWGQ3X2JNVERtQVZJcnR2eTF2ekFPMWh5TERkeXhYdU5qTHNnSXBfVzBrWFZhT1liamQ1dWFxOC1WeEpYbjJlUDNQQlZ0ZjctRVpnZkJsVUExQktlYVp4SmV2ZGpiblE4QQ?oc=5',
  'content': 'AI가 내린 결정 때문에 일어난 사상 첫 민족청소운동 - 한겨레21'},
 {'url': 'https://news.google.com/rss/articles/CBMiYEFVX3lxTE9YTkNDXzJjVExlbF9uclZUaURCRTI3OU9KNWVBX21VNHJjNV9OMEJraFZXTGxkZFBfVTVacExCLWJpam1nVnE4dzEyTkk3a0lTRTMwRGVHWWY1cEtneDZoZw?oc=5',
  'content': '카카오, AI 비서 ‘카나나’ 공개…‘카나’는 톡방 요약, ‘나나’는 일정 알림 - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE1vVU1qSUFJd3FWVV8zX193azlDVGNveUlQMnZQRU1yOVphVHl0Y1VUNU9IVDJDWDZZSUhTaExTLXl6cUJuYzZSYk1FX0c5QQ?oc=5',
  'content': '[SW와 디지털 전환] 국경없는 AI시대, 정책으로 대한민국 AI기술 글로벌 경쟁력을 확보 - 전자신문'},
 {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTFAwVk53cnB6Z3dOMEprUUVOSWpYQ0NNaFc3VTVnSk5IOWNIQlBzOUtxcjUwbG9SVTNaXzhqclhyaUF6bl9xdEdiMWVxeHVsTFJoNGlHdGx5cW4wS2puRUVXbDFHOXhoYU9rRnc?oc=5',
  'content': '애플, 내부 조사 결과 "오픈AI에 2년 

In [19]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict

@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(keyword=query, k=5)

In [20]:
# 실행 결과
search_keyword.invoke({"query": "AI"})

AI


[{'url': 'https://news.google.com/rss/articles/CBMib0FVX3lxTE4yYkp3OHJwWGQ3X2JNVERtQVZJcnR2eTF2ekFPMWh5TERkeXhYdU5qTHNnSXBfVzBrWFZhT1liamQ1dWFxOC1WeEpYbjJlUDNQQlZ0ZjctRVpnZkJsVUExQktlYVp4SmV2ZGpiblE4QQ?oc=5',
  'content': 'AI가 내린 결정 때문에 일어난 사상 첫 민족청소운동 - 한겨레21'},
 {'url': 'https://news.google.com/rss/articles/CBMiYEFVX3lxTE9YTkNDXzJjVExlbF9uclZUaURCRTI3OU9KNWVBX21VNHJjNV9OMEJraFZXTGxkZFBfVTVacExCLWJpam1nVnE4dzEyTkk3a0lTRTMwRGVHWWY1cEtneDZoZw?oc=5',
  'content': '카카오, AI 비서 ‘카나나’ 공개…‘카나’는 톡방 요약, ‘나나’는 일정 알림 - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTFAwVk53cnB6Z3dOMEprUUVOSWpYQ0NNaFc3VTVnSk5IOWNIQlBzOUtxcjUwbG9SVTNaXzhqclhyaUF6bl9xdEdiMWVxeHVsTFJoNGlHdGx5cW4wS2puRUVXbDFHOXhoYU9rRnc?oc=5',
  'content': '애플, 내부 조사 결과 "오픈AI에 2년 뒤져"...AI로 빛 보려면 몇년 더 걸려 - AI타임스'},
 {'url': 'https://news.google.com/rss/articles/CBMidEFVX3lxTE4yUnFMb1l4OWJMVWhQVlJJalFsMDctVzlpWWJUVThyYTg5bTRCVWlQT2JCXzhrY201UXBpZUI3UkJjeXBkQ3UzdjBfLU5BUERIU09QSGZMbk9SUUprRS15VGdNODZ5SlhlT0hqYl8ya1JuaH